In [1]:
# Install package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install mysql-connector-python
import mysql.connector
import json
import csv
from tqdm import tqdm

In [2]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="915997582"
)

print(db)

cursor = db.cursor()

In [3]:
cursor.execute("DROP DATABASE EDDB")

<h1>Create EDDB Database</h1>

In [4]:
cursor.execute("CREATE DATABASE EDDB")

cursor.execute("SHOW DATABASES")

for x in cursor:
  print(x)

('EDDB',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


In [5]:
db = mysql.connector.connect(
  host="localhost",
  user="jake",
  password="915997582",
  database="EDDB"  
)

print(db)

cursor = db.cursor()

<h1>Table Creation statements</h1>

In [6]:
# SYSTEMS TABLE
create_statement = """
CREATE TABLE system_populated (
id INT,
edsm_id INT,
name VARCHAR(255),
x FLOAT,
y FLOAT,
z FLOAT,
population BIGINT,
is_populated TINYINT,
government_id INT,
government VARCHAR(255),
allegiance_id INT,
allegiance VARCHAR(255),
security_id INT,
security VARCHAR(255),
primary_economy_id INT,
primary_economy VARCHAR(255),
power VARCHAR(255),
power_state VARCHAR(255),
power_state_id INT,
needs_permit TINYINT,
simbad_ref VARCHAR(255),
controlling_minor_faction_id INT,
controlling_minor_faction VARCHAR(255),
reserve_type_id INT,
reserve_type VARCHAR(255),
ed_system_address BIGINT,
PRIMARY KEY (id)
)
"""

cursor.execute(create_statement)

In [7]:
# LISTINGS CATEGORY TABLE
create_statement = """
CREATE TABLE category (
id INT,
name VARCHAR(255),
PRIMARY KEY (id)
)
"""

cursor.execute(create_statement)

In [8]:
# SYSTEMS/STATION STATES TABLE
create_statement = """
CREATE TABLE state (
id INT,
name VARCHAR(255),
PRIMARY KEY (id)
)
"""

cursor.execute(create_statement)

In [9]:
# COMMODITY TABLE
create_statement = """
CREATE TABLE commodity (
id INT,
name VARCHAR(255),
category_id INT,
average_price INT,
is_rare INT,
max_buy_price INT,
max_sell_price INT,
min_buy_price INT,
min_sell_price INT,
buy_price_lower_average INT,
sell_price_upper_average INT,
is_non_marketable INT,
ed_id INT,
PRIMARY KEY (id),
FOREIGN KEY (category_id) REFERENCES category(id)
)
"""

cursor.execute(create_statement)

In [10]:
# STATION TABLE
create_statement = """
CREATE TABLE station (
id INT,
name VARCHAR(255),
system_id INT,
max_landing_pad_size VARCHAR(255),
distance_to_star INT,
government_id INT,
government VARCHAR(255),
allegiance_id INT,
allegiance VARCHAR(255),
type_id INT,
type VARCHAR(255),
has_blackmarket TINYINT,
has_market TINYINT,
has_refuel TINYINT,
has_repair TINYINT,
has_rearm TINYINT,
has_outfitting TINYINT,
has_shipyard TINYINT,
has_docking TINYINT,
has_commodities TINYINT,
is_planetary TINYINT,
settlement_size_id INT,
settlement_size VARCHAR(255),
settlement_security_id INT,
settlement_security VARCHAR(255),
body_id INT,
controlling_minor_faction_id INT,
ed_market_id BIGINT,
PRIMARY KEY (id)
)
"""

cursor.execute(create_statement)

In [11]:
# LISTINGS TABLE
create_statement = """
CREATE TABLE listing (
id INT,
station_id INT,
supply INT,
supply_bracket INT,
buy_price INT,
sell_price INT,
demand INT,
demand_bracket INT,
PRIMARY KEY (id)
)
"""

cursor.execute(create_statement)

In [12]:
# FACTION TABLE
create_statement = """
CREATE TABLE faction (
id INT,
name VARCHAR(255),
government_id INT,
government VARCHAR(255),
allegiance_id INT,
allegiance VARCHAR(255),
home_system_id INT,
is_player_faction INT,
PRIMARY KEY (id)
)
"""

cursor.execute(create_statement)

In [13]:
# MODULE GROUP TABLE
create_statement = """
CREATE TABLE _group (
id INT,
name VARCHAR(255),
category VARCHAR(255),
PRIMARY KEY (id)
)
"""

cursor.execute(create_statement)

In [14]:
# MODULE TABLE
create_statement = """
CREATE TABLE module (
id INT,
group_id INT,
class INT,
rating VARCHAR(255),
price INT,
weapon_mode VARCHAR(255),
missile_type VARCHAR(255),
name VARCHAR(255),
belongs_to INT,
ed_id INT,
ed_symbol VARCHAR(255),
game_context_id INT,
ship VARCHAR(255),
PRIMARY KEY (id),
FOREIGN KEY (group_id) REFERENCES _group(id)
)
"""

cursor.execute(create_statement)

In [15]:
# SYSTEMS MINOR FACTION PRESENCES ** NEED to narrow down primary key
create_statement = """
CREATE TABLE minor_faction_presence (
system_id INT,
happiness_id INT,
minor_faction_id INT,
influence FLOAT,
PRIMARY KEY (system_id, minor_faction_id),
FOREIGN KEY (minor_faction_id) REFERENCES faction(id),
FOREIGN KEY (system_id) REFERENCES system_populated(id)
)
"""

cursor.execute(create_statement)

In [16]:
# STATION IMPORTS TABLE
create_statement = """
CREATE TABLE imports (
station_id INT,
commodity_id INT,
PRIMARY KEY (station_id, commodity_id),
FOREIGN KEY (station_id) REFERENCES station(id),
FOREIGN KEY (commodity_id) REFERENCES commodity(id)
)
"""

cursor.execute(create_statement)

In [17]:
# STATION EXPORTS TABLE
create_statement = """
CREATE TABLE exports (
station_id INT,
commodity_id INT,
PRIMARY KEY (station_id, commodity_id),
FOREIGN KEY (station_id) REFERENCES station(id),
FOREIGN KEY (commodity_id) REFERENCES commodity(id)
)
"""

cursor.execute(create_statement)

In [18]:
# STATION PROHIBITED TABLE
create_statement = """
CREATE TABLE prohibited (
station_id INT,
commodity_id INT,
PRIMARY KEY (station_id, commodity_id),
FOREIGN KEY (station_id) REFERENCES station(id),
FOREIGN KEY (commodity_id) REFERENCES commodity(id)
)
"""

cursor.execute(create_statement)

In [19]:
# STATION SELLING MODULES TABLE
create_statement = """
CREATE TABLE selling_module (
station_id INT,
module_id INT,
PRIMARY KEY (station_id, module_id),
FOREIGN KEY (module_id) REFERENCES module(id),
FOREIGN KEY (station_id) REFERENCES station(id)
)
"""

cursor.execute(create_statement)

<h1>Add Foreign key contraints</h1>

In [20]:
# # TWO WAY SYSTEM FACTION KEYS
# key_statement = """
# ALTER TABLE system_populated
# ADD FOREIGN KEY (controlling_minor_faction_id) REFERENCES faction(id)
# """

# cursor.execute(key_statement)

# key_statement = """
# ALTER TABLE faction
# ADD FOREIGN KEY (home_system_id) REFERENCES system_populated(id)
# """

# cursor.execute(key_statement)

<h1>Data Insert</h1>
<h1>Commodities.json</h1>

In [21]:
file = "../data/commodities.json"
data = []
with open(file) as openfile:
    data = json.load(openfile)
    
category_ids = set()

for x in tqdm(data, desc="Importing Category data"):
    if not x['category']['id'] in category_ids:
        category_ids.add(x['category']['id'])
        insert_statement = """
        INSERT INTO `category` (`id`, `name`) 
        VALUES ({},'{}')
        """.format(x['category']['id'], x['category']['name'])
        cursor.execute(insert_statement)
        db.commit()

for x in tqdm(data, desc="Importing commodity data"):
    for y in x:
        if f"{x[y]}" == "None":
            x[y] = 'NULL'

    insert_statement = """
    INSERT INTO `commodity` (`id`, `name`, `category_id`, `average_price`, `is_rare`, `max_buy_price`, `max_sell_price`, `min_buy_price`, `min_sell_price`, `buy_price_lower_average`, `sell_price_upper_average`, `is_non_marketable`, `ed_id`) 
    VALUES ({},"{}",{},{},{},{},{},{},{},{},{},{},{})
    """.format(x['id'],x['name'],x['category_id'],x['average_price'],x['is_rare'],x['max_buy_price'],x['max_sell_price'],x['min_buy_price'],x['min_sell_price'],x['buy_price_lower_average'],x['sell_price_upper_average'],x['is_non_marketable'],x['ed_id'])
    cursor.execute(insert_statement)
    db.commit()
    


Importing commodity data: 100%|██████████| 373/373 [00:02<00:00, 160.98it/s]


<h1>factions.json</h1>

In [22]:
file = "../data/factions.json"
data = []
with open(file) as openfile:
    data = json.load(openfile)
    
val = []
    
for x in tqdm(data, desc="Formatting Faction data"):
    for y in x:
        if f"{x[y]}" == "None":
            x[y] = -1
        if f"{x[y]}" == "False":
            x[y] = 0
        if f"{x[y]}" == "True":
            x[y] = 1
    val.append((x['id'], x['name'], x['government_id'], x['government'], x['allegiance_id'], x['allegiance'], x['home_system_id'], x['is_player_faction']))

sql = "INSERT INTO `faction` (`id`, `name`, `government_id`, `government`, `allegiance_id`, `allegiance`, `home_system_id`, `is_player_faction`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(sql, val)    
db.commit()
print("Commited")
    


Formatting Faction data: 100%|██████████| 77250/77250 [00:00<00:00, 202755.25it/s]


Commited


In [23]:
file = "../data/modules.json"
data = []
with open(file) as openfile:
    data = json.load(openfile)
    
val = []
group_ids = set()

for x in tqdm(data, desc="Formatting group data"):
    if x['group']['id'] not in group_ids:
        group_ids.add(x['group']['id'])
        val.append((x['group']['id'], x['group']['name'], x['group']['category']))

sql = "INSERT INTO `_group` (`id`, `name`, `category`) VALUES (%s,%s,%s)"

cursor.executemany(sql, val)    
db.commit()

val = []
    
for x in tqdm(data, desc="Formatting module data"):
    for y in x:
        if f"{x[y]}" == "None":
            x[y] = 0
        if f"{x[y]}" == "False":
            x[y] = 0
        if f"{x[y]}" == "True":
            x[y] = 1
    val.append((x['id'], x['group_id'], x['class'], x['rating'], x['price'], x['weapon_mode'], x['missile_type'], x['name'], x['belongs_to'], x['ed_id'], x['ed_symbol'], x['game_context_id'], x['ship']))

sql = "INSERT INTO `module` (`id`, `group_id`, `class`, `rating`, `price`, `weapon_mode`, `missile_type`, `name`, `belongs_to`, `ed_id`, `ed_symbol`, `game_context_id`, `ship`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(sql, val)    
db.commit()
print("Commited")

Formatting module data: 100%|██████████| 1043/1043 [00:00<00:00, 80233.64it/s]

Commited


In [24]:
file = "../data/systems_populated.json"
data = []
with open(file) as openfile:
    data = json.load(openfile)
    
val = []
    
for x in tqdm(data, desc="Formatting system data"):
    for y in x:
        if f"{x[y]}" == "None":
            x[y] = -1
        if f"{x[y]}" == "False":
            x[y] = 0
        if f"{x[y]}" == "True":
            x[y] = 1
    val.append((x['id'], x['edsm_id'], x['name'], x['x'], x['y'], x['z'], x['population'], x['is_populated'], x['government_id'], x['government'], x['allegiance_id'], x['allegiance'], x['security_id'], x['security'], x['primary_economy_id'], x['primary_economy'], x['power'], x['power_state'], x['power_state_id'], x['needs_permit'], x['simbad_ref'], x['controlling_minor_faction_id'], x['controlling_minor_faction'], x['reserve_type_id'], x['reserve_type'], x['ed_system_address']))

sql = "INSERT INTO `system_populated` (`id`, `edsm_id`, `name`, `x`, `y`, `z`, `population`, `is_populated`, `government_id`, `government`, `allegiance_id`, `allegiance`, `security_id`, `security`, `primary_economy_id`, `primary_economy`, `power`, `power_state`, `power_state_id`, `needs_permit`, `simbad_ref`, `controlling_minor_faction_id`, `controlling_minor_faction`, `reserve_type_id`, `reserve_type`, `ed_system_address`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(sql, val)    
db.commit()
print("Commited")

Formatting system data: 100%|██████████| 20569/20569 [00:01<00:00, 15512.10it/s]


Commited


In [25]:
file = "../data/stations.json"
data = []
with open(file) as openfile:
    data = json.load(openfile)
    
# Need a commodity name to id map because of how the data is structured
commodity_map = {}

map_query = "SELECT id, name FROM commodity"
cursor.execute(map_query)
results = cursor.fetchall()

for commodity in results:
    commodity_map[commodity[1]] = commodity[0]
    
# STATION
val = []
imports = []
exports = []
prohibited = []
selling_modules = []
    
for x in tqdm(data, desc="Formatting station data"):
    for y in x:
        if f"{x[y]}" == "None":
            x[y] = -1
        if f"{x[y]}" == "False":
            x[y] = 0
        if f"{x[y]}" == "True":
            x[y] = 1
    imports.append((x['id'], x['import_commodities']))
    exports.append((x['id'], x['export_commodities']))
    prohibited.append((x['id'], x['prohibited_commodities']))
    selling_modules.append((x['id'], x['selling_modules']))
    val.append((x['id'], x['name'], x['system_id'], x['max_landing_pad_size'], x['distance_to_star'], x['government_id'], x['government'], x['allegiance_id'], x['allegiance'], x['type_id'], x['type'], x['has_blackmarket'], x['has_market'], x['has_refuel'], x['has_repair'], x['has_rearm'], x['has_outfitting'], x['has_shipyard'], x['has_docking'], x['has_commodities'], x['is_planetary'], x['settlement_size_id'], x['settlement_size'], x['settlement_security_id'], x['settlement_security'], x['body_id'], x['controlling_minor_faction_id'], x['ed_market_id']))

sql = "INSERT INTO `station` (`id`, `name`, `system_id`, `max_landing_pad_size`, `distance_to_star`, `government_id`, `government`, `allegiance_id`, `allegiance`, `type_id`, `type`, `has_blackmarket`, `has_market`, `has_refuel`, `has_repair`, `has_rearm`, `has_outfitting`, `has_shipyard`, `has_docking`, `has_commodities`, `is_planetary`, `settlement_size_id`, `settlement_size`, `settlement_security_id`, `settlement_security`, `body_id`, `controlling_minor_faction_id`, `ed_market_id`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(sql, val)    
db.commit()

# IMPORTS
val = []

for x in tqdm(imports, desc="Formatting imports"):
    for commodity in x[1]:
        val.append((x[0], commodity_map[commodity]))
        
sql = "INSERT INTO `imports` (`station_id`, `commodity_id`) VALUES (%s, %s)"
cursor.executemany(sql, val)    
db.commit()  

#EXPORTS
val = []

for x in tqdm(exports, desc="Formatting exports"):
    for commodity in x[1]:
        val.append((x[0], commodity_map[commodity]))
        
sql = "INSERT INTO `exports` (`station_id`, `commodity_id`) VALUES (%s, %s)"
cursor.executemany(sql, val)    
db.commit()

#PROHIBITED
val = []

for x in tqdm(prohibited, desc="Formatting prohibited"):
    for commodity in x[1]:
        val.append((x[0], commodity_map[commodity]))
        
sql = "INSERT INTO `prohibited` (`station_id`, `commodity_id`) VALUES (%s, %s)"
cursor.executemany(sql, val)    
db.commit()

# SELLING MODULES
val = []

for x in tqdm(selling_modules, desc="Formatting selling modules"):
    for module_id in x[1]:
        val.append((x[0], module_id))
        
sql = "INSERT INTO `selling_module` (`station_id`, `module_id`) VALUES (%s, %s)"

def chunk(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in tqdm(range(0, len(lst), n), desc="Chunking..."):
        yield lst[i:i + n]

chunks = list(chunk(val, 100000))
        
for x in tqdm(chunks, desc="Uploading chunked modules data"):
    cursor.executemany(sql, x) 
    db.commit()
    
print("Commited")


Uploading chunked modules data:  50%|████▉     | 57/115 [02:29<02:32,  2.62s/it]


KeyboardInterrupt: 

In [ ]:
# import listing data that is in csv file
    
chunk_size = 1000

data_dict = {}
make_json(data_dict, '../data/listings.csv')


data = []
for key in tqdm(data_dict, desc="Converting data to correct list of JSON..."):
    dat.append(data_dict[key])
    
val = []
    
for x in tqdm(data, desc="Formatting system data"):
    for y in x:
        if f"{x[y]}" == "None":
            x[y] = -1
        if f"{x[y]}" == "False":
            x[y] = 0
        if f"{x[y]}" == "True":
            x[y] = 1
    val.append((x['id'], x['edsm_id'], x['name'], x['x'], x['y'], x['z'], x['population'], x['is_populated'], x['government_id'], x['government'], x['allegiance_id'], x['allegiance'], x['security_id'], x['security'], x['primary_economy_id'], x['primary_economy'], x['power'], x['power_state'], x['power_state_id'], x['needs_permit'], x['simbad_ref'], x['controlling_minor_faction_id'], x['controlling_minor_faction'], x['reserve_type_id'], x['reserve_type'], x['ed_system_address']))

sql = "INSERT INTO `listings` (`id`, `edsm_id`, `name`, `x`, `y`, `z`, `population`, `is_populated`, `government_id`, `government`, `allegiance_id`, `allegiance`, `security_id`, `security`, `primary_economy_id`, `primary_economy`, `power`, `power_state`, `power_state_id`, `needs_permit`, `simbad_ref`, `controlling_minor_faction_id`, `controlling_minor_faction`, `reserve_type_id`, `reserve_type`, `ed_system_address`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            
def chunk(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in tqdm(range(0, len(lst), n), desc="Chunking..."):
        yield lst[i:i + n]

chunks = list(chunk(val, 1000000))
        
for chunk in tqdm(chunks, desc="Uploading Chunks"):
    cursor.executemany(sql, chunk)
    
db.commit()
print("Commited")